In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

from nlinec.predict import predict_heads, predict_probabilities, predict_type

/home/psaegert/miniconda3/envs/fsem/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
input_ids = tokenizer('input text', return_tensors="pt", add_special_tokens=True)['input_ids'][0]
input_ids

tensor([    0, 46797,  2788,     2])

In [4]:
# Decode the tokens of the sequence
tokenizer.decode(input_ids)

'<s>input text</s>'

In [5]:
# roberta uses <s> and </s> as special tokens

In [6]:
input_ids = tokenizer('input text</s><s>another text', return_tensors="pt", add_special_tokens=True)['input_ids'][0]
input_ids

tensor([    0, 46797,  2788,     2,     0, 30303,  2788,     2])

In [7]:
premise = 'President Obama is speaking at the White House.'

In [8]:
input_text = [
    f'{premise}' + '</s><s>' + 'Barack Obama is a president.',
    f'{premise}' + '</s><s>' + 'Mike bought a new car yesterday.',
    f'{premise}' + '</s><s>' + 'Barack Obama is speaking at the Black House.'
]

In [9]:
predict_heads(model, tokenizer, input_text)

array([[2.7289167e-03, 2.3182165e-02, 9.7408891e-01],
       [5.5945998e-01, 3.7343305e-01, 6.7107014e-02],
       [9.9855858e-01, 9.4705133e-04, 4.9432134e-04]], dtype=float32)

In [10]:
predict_probabilities(model, tokenizer, sentence=[
    'President Obama is speaking at the White House.',
    'The farmer John talked about has a donkey named Fred.'
], entity=[
    'Obama',
    'Fred'
], types=['person', 'place', 'thing', 'animal', 'other', 'political figure'])

array([[[0.02397741, 0.04429863, 0.93172395],
        [0.0764203 , 0.12303857, 0.80054116],
        [0.03229134, 0.09190603, 0.87580258],
        [0.97510999, 0.01856656, 0.00632342],
        [0.47939527, 0.13846022, 0.38214445],
        [0.00402811, 0.14204757, 0.85392439]],

       [[0.98007125, 0.01074546, 0.00918332],
        [0.27966234, 0.25505066, 0.465287  ],
        [0.00373707, 0.07182994, 0.92443299],
        [0.00322229, 0.02629043, 0.9704873 ],
        [0.17918728, 0.28542724, 0.53538549],
        [0.59493619, 0.39796174, 0.00710202]]])

In [11]:
predict_probabilities(model, tokenizer, sentence=[
    'President Obama is speaking at the White House.',
    'The farmer John talked about has a donkey named Fred.'
], entity=[
    'Obama',
    'Fred'
], types=['person', 'place', 'thing', 'animal', 'other', 'political figure'],
normalize='mean')

array([[[0.01506855, 0.07934306, 0.24197731],
        [0.04802616, 0.2203738 , 0.20790793],
        [0.02029342, 0.16461247, 0.22745402],
        [0.61280559, 0.03325447, 0.00164225],
        [0.30127483, 0.24799545, 0.09924644],
        [0.00253146, 0.25442074, 0.22177205]],

       [[0.48023489, 0.0102601 , 0.00315374],
        [0.13703454, 0.24353034, 0.15978931],
        [0.00183116, 0.06858548, 0.31746967],
        [0.00157892, 0.02510293, 0.33328569],
        [0.08780176, 0.27253485, 0.1838626 ],
        [0.29151872, 0.3799863 , 0.00243898]]])

In [12]:
predict_probabilities(model, tokenizer, sentence=[
    'President Obama is speaking at the White House.',
    'The farmer John talked about has a donkey named Fred.'
], entity=[
    'Obama',
    'Fred'
], types=['person', 'place', 'thing', 'animal', 'other', 'political figure'],
normalize='softmax')

array([[[0.12183367, 0.1585615 , 0.21180222],
        [0.12839349, 0.17155132, 0.18576271],
        [0.12285081, 0.16629278, 0.20028303],
        [0.31538351, 0.15453343, 0.0839526 ],
        [0.19211125, 0.17421743, 0.12225065],
        [0.11942726, 0.17484353, 0.19594879]],

       [[0.29595798, 0.13997024, 0.09633141],
        [0.1469083 , 0.17870475, 0.15200267],
        [0.11148425, 0.14878678, 0.24057794],
        [0.11142687, 0.14216307, 0.25191668],
        [0.13286499, 0.18421648, 0.16304017],
        [0.2013576 , 0.20615867, 0.09613112]]])

In [13]:
predict_type(model, tokenizer, sentence=[
    'President Obama is speaking at the White House.',
    'The farmer John talked about has a donkey named Fred.'
], entity=[
    'Obama',
    'Fred'
], types=['person', 'place', 'thing', 'animal', 'other', 'political figure'])

array(['person', 'animal'], dtype='<U6')